In [1]:
import sys

sys.path.append('/home/ajliang/search/search')
import argparse
import torch

from utils.common_utils import load_model
from map2map.map2map.data.fields import FieldDataset

In [15]:
"""Load Dataset"""
device = torch.device("cpu")

args = argparse.Namespace(
    init_input_path="/user_data/ajliang/Linear/val/LH0045/4/dis.npy",
    style_path="/user_data/ajliang/Linear/val/LH0045/4/params.npy",
    target_output_path="/user_data/ajliang/Nonlinear/val/LH0045/4/dis.npy",
    load_backward_model_state="/home/ajliang/search/model_weights/backward_model.pt",
    verbose=False,
    dropout_prob=0.0,
    crop=128,
)

dataset = FieldDataset(
    style_pattern=args.style_path,
    in_patterns=[args.init_input_path],
    tgt_patterns=[args.target_output_path],
    # in_norms=['cosmology.dis'],
    callback_at='/home/ajliang/search/search/map2map',
    crop=args.crop,
)

In [16]:
data = dataset[0]
input = data['input']
# add batch dimension
input = input.unsqueeze(0)

In [17]:
style = data['style']
# add batch dimension
style = style.unsqueeze(0)

In [18]:
args.style_size = dataset.style_size
args.in_chan = dataset.in_chan
args.out_chan = dataset.tgt_chan
backward_model = load_model(args, device=device, path_to_model_state=args.load_backward_model_state)

Loaded model from /home/ajliang/search/model_weights/backward_model.pt, which was trained for 128 epochs.


In [19]:
backward_model(input, style)

tensor([[[[[-7.5098e+00, -6.5686e+00, -5.6308e+00,  ..., -5.2897e+00,
            -5.2921e+00, -5.1096e+00],
           [-6.0248e+00, -6.1081e+00, -5.3485e+00,  ..., -4.2572e+00,
            -4.0158e+00, -4.0710e+00],
           [-5.1805e+00, -4.7929e+00, -4.0245e+00,  ..., -2.3980e+00,
            -1.6634e+00, -2.2212e+00],
           ...,
           [-4.8650e+00, -5.8876e+00, -6.6499e+00,  ..., -4.7987e+00,
            -4.6832e+00, -5.3960e+00],
           [-7.7215e+00, -8.7801e+00, -8.7646e+00,  ..., -2.6204e+00,
            -1.4767e+00, -3.0194e+00],
           [-1.0854e+01, -1.1609e+01, -1.0407e+01,  ..., -2.1948e+00,
            -2.1315e+00, -2.6850e+00]],

          [[-7.3014e+00, -6.5851e+00, -5.4688e+00,  ..., -5.5064e+00,
            -6.7908e+00, -5.8012e+00],
           [-6.0289e+00, -5.4169e+00, -5.5505e+00,  ..., -4.7392e+00,
            -5.8808e+00, -4.9066e+00],
           [-5.3995e+00, -4.9443e+00, -4.5017e+00,  ..., -2.1419e+00,
            -3.4565e+00, -3.4205e+00],
 

In [20]:
_.shape

torch.Size([1, 3, 32, 32, 32])

In [33]:
import torch
from map2map.map2map.models import StyledVNet

# x has shape (batch_size, in_chan, x, y, z)
x = torch.randn(1, 3, 128, 128, 128)

model: StyledVNet = StyledVNet(
    style_size=1,
    in_chan=3,
    out_chan=3,
    dropout_prob=0.0,
)
style = torch.randn(1, 1)
output = model(x, style)

# output has shape (batch_size, out_chan, x - 96, y - 96, z - 96)
# Why? Is it possible to NOT truncate the X, Y, Z dimensions of the output? i.e. let the output have the same shape as the input?
assert output.shape == torch.Size([1, 3, 32, 32, 32])

In [32]:
output.shape

torch.Size([1, 3, 32, 32, 32])

In [27]:
style = torch.randn(1, 1)